In [ ]:
# Testing Clifford tester on Quantum Inspire
from qiskit import QuantumCircuit
from qiskit_quantuminspire.qi_provider import QIProvider

from lib.clifford_tester import clifford_tester_paired_runs
from lib.clifford_tester.results import summarise_paired
from lib.qi_transpilation import get_backend_and_transpilation_function


def acceptance_rate(U, n, shots=20, backend=None, transpilation_function=None):
    return summarise_paired(clifford_tester_paired_runs(U, n, shots=shots, backend=backend, transpilation_function=transpilation_function))


# Connect to Quantum Inspire
provider = QIProvider()
print("Available backends:")
for backend in provider.backends():
    print(f"  {backend.name}")

# https://monitoring.qutech.support/public-dashboards/c494a21fb6b7405f850ab8f340f798ef?orgId=1&refresh=10s&from=now-2d&to=now

In [3]:
# Select a backend (use a simulator backend for testing, or a real QPU)
backend, transpilation_function = get_backend_and_transpilation_function("Tuna-9")
print(f"Using backend: {backend.name}")

Using backend: Tuna-9


In [ ]:
# Test Clifford gate: Hadamard (should accept with high probability)
H = QuantumCircuit(1)
H.h(0)
h_acceptance = acceptance_rate(H, 1, backend=backend, transpilation_function=transpilation_function)
print(f"Hadamard (Clifford) acceptance rate: {h_acceptance:.2f}")

In [ ]:
# Test non-Clifford gate: T gate (should reject with noticeable probability)
T = QuantumCircuit(1)
T.t(0)
t_acceptance = acceptance_rate(T, 1, backend=backend, transpilation_function=transpilation_function)
print(f"T gate (non-Clifford) acceptance rate: {t_acceptance:.2f}")

In [ ]:
# 4-qubit Clifford circuit (uses 8 of Tuna-9's 9 qubits for the tester)
# H gates on all qubits + CNOT chain - should accept with high probability
clifford_4q = QuantumCircuit(4)
for i in range(4):
    clifford_4q.h(i)
for i in range(3):
    clifford_4q.cx(i, i + 1)

print("4-qubit Clifford circuit:")
print(clifford_4q.draw())

clifford_4q_acceptance = acceptance_rate(clifford_4q, 4, backend=backend, transpilation_function=transpilation_function)
print(f"\n4-qubit Clifford acceptance rate: {clifford_4q_acceptance:.2f}")

In [ ]:
# 4-qubit non-Clifford circuit (uses 8 of Tuna-9's 9 qubits for the tester)
# T gates on all qubits - should reject with noticeable probability
non_clifford_4q = QuantumCircuit(4)
for i in range(4):
    non_clifford_4q.t(i)

print("4-qubit non-Clifford circuit (T gates):")
print(non_clifford_4q.draw())

non_clifford_4q_acceptance = acceptance_rate(non_clifford_4q, 4, backend=backend, transpilation_function=transpilation_function)
print(f"\n4-qubit non-Clifford acceptance rate: {non_clifford_4q_acceptance:.2f}")